In [4]:
import os
import pandas as pd
import glob
import re
import pickle

os.chdir("..")


In [5]:

# Define the directory path where the CSV files are located
directory_path = '/home/james/programming_projects/work/SDG_11.2.1/data/population_estimates/2002-2012'

# Get a list of all CSV files in the directory
file_list = glob.glob(f'{directory_path}/*.csv')

# Initialize empty dictionaries to store the dataframes
all_ages_df = {}
male_df = {}
female_df = {}

years = ["2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012"]




In [6]:
def extract_region(file_name):
    # Extact the region name from the file name
    region_pattern = r"unformatted-(.*?)-mid2002"
    match = re.search(region_pattern, file_name)

    if match:
        region = match.group(1)

    return region

regions = [extract_region(file) for file in file_list]
    
    

In [7]:
regions

['London',
 'South-West',
 'South-East',
 'Wales',
 'East-Mids',
 'North-West',
 'Yorks-and-H',
 'West-Mids',
 'North-East']

In [8]:

def age_pop_one_year(df, sex_num, year):
    """Gets the population for each age in every OA in a region for one year.capitalize
    
    Supply sex_num, 1 for male and 2 for female, None for both."""
    
    if sex_num:
        sex_mask = df.Sex==sex_num
        # get single sex df
        df = df[sex_mask]
    # drop sex column
    cols = ["OA11CD", "Age", "LAD11CD", f"Population_{year}"]
    df = df[cols]
    grouped_df = df.groupby(["OA11CD", "Age"]).sum().pivot_table(index="OA11CD", columns="Age", values="Population_2002")
    grouped_df["All Ages"] = grouped_df.sum(axis=1)
    
    return grouped_df
    

In [11]:
def load_pickle_file(pickle_path, csv_path, year):
    persistent_exists = os.path.isfile(pickle_path)
    if persistent_exists:
        with open(pickle_path, "rb") as pickle_file:
            df = pickle.load(pickle_file)
            df = df[year]
            print("Pickle file loaded successfully.")
    else:
        # Read the CSV file into a dataframe
        df = pd.read_csv(csv_path)
    return df, persistent_exists


dfs_dict = {}
        
def proc_all_regions_by_year(file_path, dfs_dict, year, pickle_file_path):
    """Processes the file for each region for a given year."""
    
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)
    
    # Get the data for each year
    print(f"Processing {year}")
        
    # Extract the region name from the file name
    region_pattern = r"unformatted-(.*?)-mid2002"
    match = re.search(region_pattern, file_path)

    if match:
        region = match.group(1)
        print(f"Processing {region}")
    
        # Check if there is a region key in the dict, if not initialize an empty dictionary for the region
        year_key = f"{year}_data"
        if year_key not in dfs_dict:
            dfs_dict.setdefault(year_key, {})
        
        # # Load pickle if it exists or CSV if not
        df, _ = load_pickle_file(pickle_file_path, file_path, year)

        year_data_col = f"Population_{year}"
        
        # Subset the dataframe to get the data for the year
        cols = ["OA11CD", "Sex", "Age", "LAD11CD", year_data_col]
        year_df = df[cols]
        
        # Add the data for the year to the dfs_dict[region] dictionary
        dfs_dict[year_key].update({region: year_df})
    else:
        print(f"No region found in: {file_name}.")

for file_path in file_list:
    proc_all_regions_by_year(file_path=file_path,
                             dfs_dict=dfs_dict,
                             year=2002,
                             pickle_file_path="fake/path/to/pickle/file")

# Stack the dataframes for each region for one year
dfs_dict[]


    # # dump to json
    # if not persistent_exists:    
    #     print("The pickle file didn't exist. Creating it now for faster loading next time.")
    #     with open(pickle_file_path, "wb") as pickle_file:
    #         pickle.dump(dfs_dict[region_df], pickle_file)


Processing 2002
Processing London


Processing 2002
Processing South-West
Processing 2002
Processing South-East
Processing 2002
Processing Wales
Processing 2002
Processing East-Mids
Processing 2002
Processing North-West
Processing 2002
Processing Yorks-and-H
Processing 2002
Processing West-Mids
Processing 2002
Processing North-East


In [14]:
dfs_dict['2002_data']

{'London':             OA11CD  Sex  Age    LAD11CD  Population_2002
 0        E00000001    1    0  E09000001                0
 1        E00000001    1    1  E09000001                1
 2        E00000001    1    2  E09000001                0
 3        E00000001    1    3  E09000001                2
 4        E00000001    1    4  E09000001                3
 ...            ...  ...  ...        ...              ...
 4559641  E00176595    2   86  E09000011                0
 4559642  E00176595    2   87  E09000011                0
 4559643  E00176595    2   88  E09000011                0
 4559644  E00176595    2   89  E09000011                0
 4559645  E00176595    2   90  E09000011                0
 
 [4559646 rows x 5 columns],
 'South-West':             OA11CD  Sex  Age    LAD11CD  Population_2002
 0        E00072563    1    0  E06000022                1
 1        E00072563    1    1  E06000022                1
 2        E00072563    1    2  E06000022                0
 3        E000725

In [17]:
# Concatenate the dataframes for each year, combining all the regions
year = "2002"
all_regions_by_year = {}

def concat_all_regions_same_year(dfs_dict, regions, year):
   all_regions_in_year = [dfs_dict[f"{year}_data"][region] for region in regions]
   concated_df = pd.concat(all_regions_in_year)
   return concated_df

df = concat_all_regions_same_year(dfs_dict, regions, year)

In [19]:
# check if folder exists and if not create it

pop_est_outdir = f"data/population_estimates/{year}"

if not os.path.exists(pop_est_outdir):
    os.mkdir(pop_est_outdir)

# Write the dataframes to feather files
dfs_by_year[year].reset_index().to_feather(os.path.join(pop_est_outdir, f"All_regions_{year}.feather"))

In [18]:
both_sexes_df = age_pop_one_year(df=df, sex_num=None, year=year)
male_df = age_pop_one_year(df=df, sex_num=1, year=year)
female_df = age_pop_one_year(df=df, sex_num=2, year=year)

In [19]:
female_df.head()

Age,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,All Ages
OA11CD,,,,,,,,,,,,,,,,,,,,,
E00000001,0,1,1,3,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,112
E00000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
E00000005,2,0,0,0,2,0,0,0,0,0,...,1,1,2,0,1,0,1,0,0,182
E00000007,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,81
E00000010,3,0,2,0,1,0,0,0,0,0,...,1,1,0,0,0,3,0,0,1,78


In [20]:
male_df.head()

Age,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,All Ages
OA11CD,,,,,,,,,,,,,,,,,,,,,
E00000001,0,1,0,2,3,0,0,1,0,0,...,2,1,0,0,0,0,0,1,0,138
E00000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
E00000005,3,0,2,1,2,1,0,1,1,0,...,0,0,0,0,1,0,0,0,2,187
E00000007,0,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,84
E00000010,2,0,4,3,0,1,1,0,0,0,...,2,0,1,2,0,0,0,0,2,79


In [21]:
both_sexes_df.head()

Age,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,All Ages
OA11CD,,,,,,,,,,,,,,,,,,,,,
E00000001,0,2,1,5,3,0,1,1,1,0,...,3,1,0,0,0,0,0,1,0,250
E00000003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40
E00000005,5,0,2,1,4,1,0,1,1,0,...,1,1,2,0,2,0,1,0,2,369
E00000007,0,0,1,2,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,165
E00000010,5,0,6,3,1,1,1,0,0,0,...,3,1,1,2,0,3,0,0,3,157


In [34]:
years = map(str, list(range(2002, 2013)))

In [36]:
next(years)


'2002'

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
